# **Currencies exchange rate**

In [59]:
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from bs4 import BeautifulSoup


# import matplotlib.pyplot as plt


# Class sulotion
class Currencies:

    # Initializing the number of days back, base&out currencies
    def __init__(self, daysback, base):
        self.startday = date.today() - timedelta(days=daysback)
        self.endday = date.today()
        self.base = base
        # self.out = out
        # https://api.exchangerate.host/timeseries?base={0}&start_date={1}&end_date={2}&symbols={3}
        self.url = 'https://api.exchangerate.host/timeseries?base={0}&start_date={1}&end_date={2}'.format(self.base,
                                                                                                          self.startday,
                                                                                                          self.endday)
        self.response = requests.get(self.url)
        self.data = self.response.json()
        # create dict of currencies and relevant dates
        self.currencies_data = self.data['rates']

    def all_currencies(self):
        '''c = []
        for v in self.currencies_data.values():
            for v1 in v:
                c.append(v1)
        c = set(c)
        return c'''

        page = requests.get("https://www.iban.com/currency-codes")
        soup = BeautifulSoup(page.content, 'html.parser')
        tbl1 = soup.find('table')
        headers = [th.text.strip() for th in tbl1.find('thead').find_all('th')]
        data = []

        for tr in tbl1.find('tbody').find_all('tr'):
            row_data = {}
            for i, td in enumerate(tr.find_all('td')):
                row_data[headers[i]] = td.text.strip()
            data.append(row_data)

        country_df = pd.DataFrame(data)
        return country_df

    def return_df(self):
        # creating df:
        df = pd.DataFrame.from_dict(self.currencies_data).T
        df.index.name = 'Date'
        df = df.drop(columns=['USD'])
        df = df.rename(columns={"ILS": "USD"}).round(2)
        conversion_rate = df.USD
        df.iloc[:, df.columns != 'USD'] = df.iloc[:, df.columns != 'USD'].apply(
            lambda x: (conversion_rate / x).round(2), axis=0)
        df["Base Rate"], df['ILS'] = 'ILS', 1
        self.df = df
        return df

    def return_common_curr(self):
        df2 = self.df.copy()
        df2 = df2[['USD', 'GBP', 'EUR']]
        # Calculate daily diff 
        df2["Previous_USD"] = ((df2.USD.diff() / df2.USD) * 100).round(2).fillna(0).astype(str) + '%'
        df2["Previous_GBP"] = ((df2.GBP.diff() / df2.GBP) * 100).round(2).fillna(0).astype(str) + '%'
        df2["Previous_EUR"] = ((df2.EUR.diff() / df2.EUR) * 100).round(2).fillna(0).astype(str) + '%'
        return df2

    def your_df(self, *args):
        df3 = self.df.copy()
        df3 = df3[[*args]]
        return df3

    def taxfree_check(self, price, curr):
        usd_ex_rate = self.df.at[date.today().isoformat(), 'USD']
        ex_rate = self.df.at[date.today().isoformat(), curr]
        calc_in_nis = ex_rate * price
        calc_in_usd = calc_in_nis / usd_ex_rate
        if calc_in_usd > 75:
            print("%s$ - Tax is requeried" % (calc_in_usd.round(2)))
        else:
            print("%s$ - You're good" % (calc_in_usd.round(2)))

    def highest_rate(self, curr='USD'):
        higest_curr_df = self.df[self.df[curr] == self.df[curr].max()][[curr]]
        return higest_curr_df

# data = Currencies(5, 'USD')
# df = data.return_df()
# common_currencies_df = data.return_common_curr()
# currencies_df = data.all_currencies()
# my_df = data.your_df('USD', 'EUR', 'GBP')
# print(data.taxfree_check(270, 'ILS'))
# highest_rate_df = data.highest_rate('GBP')

In [73]:
pd.read_html("https://www.iban.com/currency-codes")

[               Country         Currency Code  Number
 0          AFGHANISTAN          Afghani  AFN   971.0
 1        ÅLAND ISLANDS             Euro  EUR   978.0
 2              ALBANIA              Lek  ALL     8.0
 3              ALGERIA   Algerian Dinar  DZD    12.0
 4       AMERICAN SAMOA        US Dollar  USD   840.0
 ..                 ...              ...  ...     ...
 264  WALLIS AND FUTUNA        CFP Franc  XPF   953.0
 265     WESTERN SAHARA  Moroccan Dirham  MAD   504.0
 266              YEMEN      Yemeni Rial  YER   886.0
 267             ZAMBIA   Zambian Kwacha  ZMW   967.0
 268           ZIMBABWE  Zimbabwe Dollar  ZWL   932.0
 
 [269 rows x 4 columns]]